**Задание 1**  
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

In [2]:
movies_data = pd.read_csv('movies.csv')
ratings_data = pd.read_csv('ratings.csv')
movies_full_data = pd.merge(movies_data, ratings_data, how='left', on='movieId')
movies_full_data.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


In [3]:
def classifier(rating):
    """Классифицирует фильмы как high (4.5-5.0), medium (2.5-4.0), low (0.0-2.0)"""
    if rating == 5.0 or rating == 4.5:
        return 'high rating'
    elif rating == 4.0 or rating == 3.5 or rating == 3.0 or rating == 2.5:
        return 'middle rating'
    elif rating == 2.0 or rating == 1.5 or rating == 1.0 or rating == 0.5 or rating == 0.0:
        return 'low rating'

In [4]:
movies_full_data['class'] = movies_full_data['rating'].apply(classifier)
movies_full_data.head()

,movieId,title,genres,userId,rating,timestamp,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,middle rating
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,middle rating
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,high rating
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,middle rating
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,high rating


In [5]:
# Ради интереса посмотрим на среднюю оценку каждого фильма

movies_full_data.groupby('title').mean().sort_values('rating', ascending=False)['rating'].head(10)

title
Gena the Crocodile (1969)                    5.0
True Stories (1986)                          5.0
Cosmic Scrat-tastrophe (2015)                5.0
Love and Pigeons (1985)                      5.0
Red Sorghum (Hong gao liang) (1987)          5.0
Thin Line Between Love and Hate, A (1996)    5.0
Lesson Faust (1994)                          5.0
Eva (2011)                                   5.0
Who Killed Chea Vichea? (2010)               5.0
Siam Sunset (1999)                           5.0
Name: rating, dtype: float64

**Задание 2**  
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [6]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [7]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
            }

In [8]:
def geo_class(city):
    if city in geo_data['Центр']:
        return 'Центр'
    elif city in geo_data['Северо-Запад']:
        return 'Северо-Запад'
    elif city in geo_data['Дальний Восток']:
        return 'Дальний Восток'
    else:
        return 'undefined'

In [9]:
keywords['region'] = keywords.keyword.apply(geo_class)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [10]:
# Проверим, есть ли совпадения по Центральному региону

keywords[keywords.region == 'Центр'].head()

,keyword,shows,region
1259,москва,153050,Центр
6075,ярославль,41720,Центр
11785,тула,23898,Центр


**Задание 3 (бонусное)**  
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    1. для каждой строки пройдите по всем годам списка years
    2. если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    3. если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [11]:
movies_full_data.head()

,movieId,title,genres,userId,rating,timestamp,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,middle rating
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,middle rating
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,high rating
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,middle rating
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,high rating


In [12]:
years = list(range(1950, 2011))
print(years)

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [13]:
def production_year(title):
    check = 'no'
    for year in years:
        if str(year) in title:
            check = 'yes'
            return year
    if check == 'no':
        return '1900'

In [14]:
movies_full_data['year'] = movies_full_data.title.apply(production_year)
movies_full_data

,movieId,title,genres,userId,rating,timestamp,class,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08,middle rating,1995
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08,middle rating,1995
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09,high rating,1995
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09,middle rating,1995
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09,high rating,1995
...,...,...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09,middle rating,1900
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09,middle rating,1900
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09,middle rating,1900
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09,middle rating,1900


In [15]:
movies_full_data.groupby('year').mean().sort_values('rating', ascending=False)['rating'].head(10)

year
1957    4.039535
1954    4.009191
1962    3.969466
1952    3.953125
1972    3.944293
1964    3.940160
1974    3.935622
1967    3.922572
1975    3.879121
1958    3.842424
Name: rating, dtype: float64

In [16]:
# Результат показывает, что мнение “раньше снимали настоящее кино, не то что сейчас” соответствует действительности